### Оценка точности работы сервиса (Caila)


Для оценки точности работы сервиса загрузим датасет для проверки.

Испоьзовался датасет NERUS - https://github.com/natasha/nerus

Nerus — большой датасет русского языка , аннотированный тегами POS, синтаксическими деревьями и тегами NER (PER, LOC, ORG). Nerus имеет определенную степень ошибок в разметке, но качество довольно высокое. Корпус содержит ~700 тыс. новостных статей из Lenta.ru. Использовались инструменты из проекта Natasha: Razdel для сегментации предложений и токенов, модели Slovnet BERT для морфологии, синтаксиса и аннотации NER. Разметка хранится в стандартном формате CoNLL-U.

In [27]:
from nerus import load_nerus

#Загружаем датасет из файла
docs = load_nerus(r"C:\Users\tkav1\Downloads\nerus_lenta.conllu.gz")


texts_persons = [] #список для сущностей с ФИО
texts_list = [] #список для текстов

for i in range(100):
    
    list_pers = []

    #Проходимся по первым-i текстам
    doc = next(docs)

    text_full = doc.ner.text
    texts_list.append(text_full) #Добавляем текст в список

    #Ищем участки текста, помеченные 'PER' - указатель на ФИО
    for j in range(len(doc.ner.spans)):
        
        if doc.ner.spans[j].type == 'PER':
            person = doc.ner.text[doc.ner.spans[j].start:doc.ner.spans[j].stop]
            list_pers.append(person) #Если нашли указатель - добавляем сущность в список сущностей
             
    texts_persons.append(list_pers) #Каждый текст имеет свой список сущностей

In [28]:
#Пример
texts_list[0], texts_persons[0]

('Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.',
 ['Татьяна Голикова', 'Голиковой'])


Верные данные подготовили, теперь можно отправлять тексты с датасета на оценку на сам сервис.

В данном случае используется упрощенный код, непосредственно применяемый в сервисе. Код делает запрос к API CAILA для извлечения полных и неполных имен (ФИО) из текстов, переданных в виде списка предложений.

In [29]:
import requests
import json
import re

#Здесь задается URL для API CAILA и заголовки, включая Authorization с токеном доступа. Важно использовать свой API ключ для доступа
url = 'https://caila.io/api/adapters/openai/chat/completions'
headers = {
    'Authorization': 'API-KEY', #ВВЕДИТЕ СВОЙ API_KEY с CAILA
    'Content-Type': 'application/json'
}

#Эта функция принимает на вход список текстов data_input и возвращает список имен (ФИО), найденных в этих текстах.
def model_sample(data_input, model_name):

    names_list = []

    # Проходимся по каждому предложению в списке data_input. Каждое текст обрабатывается отдельно.
    for i in range(len(data_input)):
        
        # Данные в формате JSON
        data = {
            "texts": [data_input[i]]
        }

        # Преобразуем массив предложений в один текст для отправки в запросе
        text_input = " ".join(data["texts"])
        
        # Формируется запрос к модели gpt-3.5-turbo с инструкцией извлечь полные и неполные имена из текста.
        request_data = {
            "model": f"just-ai/openai-proxy/{model_name}",
            "messages": [
                {"role": "user", "content": f"Извлеки все имена полные и неполные ФИО следующего текста: {text_input}"}
            ]
        }
        
        # Запрос отправляется с помощью requests.post, и ответ преобразуется в JSON для дальнейшей обработки.
        response = requests.post(url, headers=headers, json=request_data)
        response_json = response.json()
        
        # Обрабатываем ответ
        try:
            #Из ответа API извлекаются данные с помощью метода .get(), чтобы избежать ошибок, если данных нет.
            choices = response_json.get('choices', [])
            if choices:
                content = choices[0].get('message', {}).get('content', '')
        
                # Регулярное выражение ищет:
                #Полные имена (две части с заглавными буквами для латиницы и кириллицы).
                #Отдельные имена (одна часть для неполных имен).
                #Имена сохраняются в переменной names.
                name_pattern = re.compile(r'\b([A-Z][a-z]+ [A-Z][a-z]+|[A-Z][a-z]+|[А-ЯЁ][а-яё]+ [А-ЯЁ][а-яё]+|[А-ЯЁ][а-яё]+)\b')
                names = name_pattern.findall(content)
            else:
                print("Нет данных в ответе.")
        except Exception as e:
            #Ловим возможные ошибки при обработке ответа от API.
            print(f"Ошибка при обработке ответа: {e}")
            
        #Найденные имена добавляются в список names_list
        names_list.append(names)
        
    return names_list

In [30]:
#Предсказываем/ищем сущности
names_gpt4_mini = model_sample(texts_list, 'gpt-4o-mini') #gpt-4o-mini
names_gpt3_5_turbo = model_sample(texts_list, 'gpt-4o-mini') #gpt-3.5-turbo

#Получили результаты предсказаний модели
names_gpt4_mini, names_gpt3_5_turbo

([['Татьяна Голикова', 'Неполных'],
  ['Вот', 'Дмитрий Любинский', 'Александр Логинов', 'Антон Шипулин', 'Если'],
  ['Упоминаются', 'Instagram', 'Диснейленд', 'Исландия'],
  ['Робер Мюллер', 'Дональд Трамп', 'Более'],
  ['Из',
   'Кристофер Доннелли',
   'Anonymous',
   'Integrity Initiative',
   'Полные',
   'Кристофером Доннелли',
   'Остальные'],
  ['Вот', 'Лонгин', 'Варфоломей', 'Филарет', 'Пётр Порошенко', 'Варфоломей'],
  ['Полные', 'Александр Бугаев'],
  ['Из',
   'Алекс Коллинс',
   'Трейси Холлоуэй',
   'Сергей Скрипаль',
   'Юлия',
   'Сергея Скрипаля'],
  ['Текст'],
  ['Имена',
   'Муталиба Эмиралиева',
   'Эмиралиев',
   'Петр Великий',
   'Полное',
   'Муталиба Эмиралиева',
   'Неполное',
   'Эмиралиев',
   'Имя',
   'Петр Великий'],
  ['Из', 'Полное', 'Тереза Мэй', 'Неполное', 'Мэй'],
  ['Александр Овечкин', 'Майкл Гартнер'],
  ['Вячеслав Бутаков', 'Также', 'Если'],
  ['Упоминаются',
   'Якутия',
   'Оренбург',
   'Приморский',
   'Мурманск',
   'Магадан',
   'Вологда',
 

### recall, presicion, f1

Искать recall, presicion, f1 будем как Macro-averaged метрики, т.е. искать метрики для каждой пары (список правильных имен и предсказанных имен) и выводит средние значения.

In [31]:
#Импортируем функцию mean из модуля statistics для вычисления средних значений по спискам.
from statistics import mean 

def metrics_eval(names):
    #Создаем три пустых списка для хранения значений Precision, Recall и F1-score для каждого текста в наборе данных.
    precision_list = []
    recall_list = []
    f1_list = []
    
    #Проходимся по каждому тексту в списке texts_persons, который содержит правильные сущности (персоны), 
    #и одновременно по соответствующим предсказаниям в списке names. Здесь предполагается, 
    #что texts_persons и names — это списки списков, где каждый элемент — это список имен для конкретного текста.
    for i in range(len(texts_persons)):
    
    
        #Преобразуем списки имен в множества (set_true и set_pred), чтобы удобно сравнивать элементы и находить пересечения.
        set_true = set(texts_persons[i])  # список правильных ответов
        set_pred = set(names[i])  # список предсказанных сущностей
    
        #Вычисление TP, FP и FN
        tp = len(set_true & set_pred)  # Пересечение
        fp = len(set_pred - set_true)  # Только в предсказанных
        fn = len(set_true - set_pred)  # Только в истинных
        
        #Вычисление Precision, Recall и F1 Score
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
        #Для каждого текста добавляем вычисленные Precision, Recall и F1-score в соответствующие списки.
        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)
    
    #Вывод средних значений метрик
    precision_mean = f"Precision (mean): {mean(precision_list)}"
    recall_mean = f"Recall (mean): {mean(recall_list)}"
    f1_score_mean = f"F1-score (mean): {mean(f1_list)}"

    return precision_mean, recall_mean, f1_score_mean

### Результаты для gpt4_mini и gpt3_5_turbo (для 100 запросов)

In [32]:
#gpt4_mini
metrics_eval(names_gpt4_mini)

('Precision (mean): 0.2621998556998557',
 'Recall (mean): 0.33906493506493507',
 'F1-score (mean): 0.28454695304695304')

In [33]:
#gpt3_5_turbo
metrics_eval(names_gpt3_5_turbo)

('Precision (mean): 0.27856593406593405',
 'Recall (mean): 0.3308982683982684',
 'F1-score (mean): 0.2865015479876161')

Готово! Имеем в виду, что рассматриваем только точные совпадения. "Робера Мюллера" != "Робер Мюллер"

### Типовые ошибки

1. Модель может предсказать только часть полного имени. Например, предсказывает "Иван", когда в оригинальном тексте "Иван Иванов".
2. Модель может путать именованные сущности с обычными словами, например, "Московский" может быть предсказано как имя, хотя в контексте оно может быть связано с географией или являться прилагательным.
3. В отношении русских фамилий модель может путать именнованные сущности с географическими названиями.
4. Если имя встречается в необычном контексте или нетипичном месте предложения, модель может его не распознать.
5. Модель может путать именованные сущности со прочими словами, начинающимися с заглавной буквы.

### Почему точность не 100% (далеко нет)

1. Модель может иметь сложности с распознаванием именованных сущностей, поскольку она не обучена специально для задачи NER.
2. Если в исходных данных или аннотациях есть ошибки или пропуски, это повлияет на результат предсказания.
3. Некоторые имена могут быть омонимами или иметь сложную структуру (например, восточные или сложные составные имена), что может приводить к ошибкам.
4. Рассматриваем только строгие совпадения.
5. Запрос по поиску сущностей требует дополнительной модификации для более точного ответа (Например, начали извлекаться "Из", если написать в запросе "из текста").